In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path
from tqdm import tqdm
import os

TODO:
- Table with metrics summary (mean + std)
- Create full plot (with multiple subplots)
- Possibly: assert to check ass exp uses same dsets

In [8]:
# Make sure the path is correct, i.e. change according which experiment we want
# root_path = Path('results_atari')
root_path = Path('results')

experiments = sorted([path.stem for path in root_path.iterdir()])
exp_box = [widgets.Checkbox(value=False, description=label, layout=widgets.Layout(width='600px')) for label in experiments]
exp_vbox = widgets.VBox(exp_box)
display(exp_vbox)

In [3]:
# OUTPUT_FOLDER = root_path / 'paper_3'
# assert not os.path.exists(OUTPUT_FOLDER)
# os.mkdir(OUTPUT_FOLDER), 'OUTPUT FOLDER ALREADY EXISTS! Delete it or change name'

In [9]:
# Load data
metrics_paths = []
for checkbox in exp_box:
    if checkbox.value == True:
        metrics_paths.append((checkbox.description, root_path / checkbox.description / 'run_plot' / 'metrics.csv'))

# Get only exp names
exps_names = [n for n, _ in metrics_paths]

# load dfs and put in a dict with exp name as key
dfs = {name: pd.read_csv(mpath) for name, mpath in metrics_paths}

In [10]:
metrics_plot = dfs[exps_names[0]]
datasets_checkboxes = [widgets.Checkbox(value=True if label == 'BUS_SELECTED' else False, description=label) for label in pd.unique(metrics_plot['dataset'])]
datasets_vbox = widgets.VBox(children=datasets_checkboxes)
 
metrics_checkboxes = [widgets.Checkbox(value=True, description=label) for label in pd.unique(metrics_plot['metric'])]
metrics_vbox = widgets.VBox(children=metrics_checkboxes)

split_checkboxes = [widgets.Checkbox(value=True, description=label) for label in ['train', 'validation', 'test']]
split_vbox = widgets.VBox(split_checkboxes)


vboxes = [datasets_vbox, metrics_vbox, split_vbox]
output = widgets.HBox(children=vboxes)
display(output)

In [11]:
# Get selected dsets
dsets = [d.description for d in datasets_checkboxes if d.value == True]
# Get selected metrics
metrics = [m.description for m in metrics_checkboxes if m.value == True]
# Get selected splits (train, validation, test)
splits = [c.description for c in split_checkboxes if c.value == True]


vals = []
# Loop over dsets
for dset_name in dsets:
    # Loop over type (train, validation, test)
    for split in ['test']:#splits:
        # Loop over dset
        for exp in exps_names:
            # Get dataframe
            df = dfs[exp]
            # Filted dset
            df = df[df['dataset'] == dset_name]
            # Select split
            df_split = df[df['split'] == split]
            # Create empty row
            row = []
            # Loop over metrics
            for metric_name in metrics:
                # Filter metrics
                df_metric = df_split[df_split['metric'] == metric_name]
#                 print(df_split[df_split['epoch'] == -1])
                df_metric = df_metric[df_metric['epoch'] == -1]
                row.append(df_metric['value'].item())
            vals.append(row)
            
#                 # Create row name
#                 row_name = f"{exp}_{dset_name}_{metric_name}"
#                 # Insert in df with new index
#                 vs.append(df_split['value'].set_axis(row_name, axis=0).rename(metric_name))
#                 values.insert(0, col_name, df_split['value'].set_axis(df_split['epoch'])) # works also
# Merge into a single df
# values = pd.concat(vs)#, axis=1)
# values
df_all = pd.DataFrame(vals, index=exps_names, columns=metrics)
df_all

,loss,accuracy,precision,recall,f1score,specificity,iou,dice,hd
20230507_bbbbd3_load_100k_7,0.702775,0.987417,0.953990,0.883812,0.917561,0.996332,0.847679,0.917561,40.709476
20230512_load_bts4_4,0.674358,0.986271,0.941090,0.881930,0.910550,0.995249,0.835789,0.910550,45.657789
20230512_load_bts5_5,0.678141,0.986664,0.957377,0.870440,0.911841,0.996665,0.837967,0.911841,65.529633
20230514_load_bts_7,0.691118,0.985374,0.924642,0.887751,0.905821,0.993774,0.827855,0.905821,93.444069
20230516_141_load_bts2_2,0.695957,0.985501,0.956570,0.855871,0.903423,0.996656,0.823857,0.903423,53.823487


In [12]:
for col_name, values in df_all.iteritems():
    mn = round(values.mean(), 4)
    std = round(values.std(), 4)
#     print(f"{col_name}: {mn} +- {std}")
    print('{:>22}'.format(col_name), '{:>12}'.format(mn), '{:>2}'.format("+-"), '{:2}'.format(std))

                  loss       0.6885 +- 0.012
              accuracy       0.9862 +- 0.0008
             precision       0.9467 +- 0.014
                recall        0.876 +- 0.0129
               f1score       0.9098 +- 0.0055
           specificity       0.9957 +- 0.0012
                   iou       0.8346 +- 0.0093
                  dice       0.9098 +- 0.0055
                    hd      59.8329 +- 21.0054


/tmp/ipykernel_14082/3605444602.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, values in df_all.iteritems():


## Last value

In [13]:
vals = []
# Loop over dsets
for dset_name in dsets:
    # Loop over dset
    for exp in exps_names:
        # Create empty row
        row = []
        cols_names = []
        # Loop over type (train, validation, test)
        for split in splits:
            # Get dataframe
            df = dfs[exp]
            # Filted dset
            df = df[df['dataset'] == dset_name]
            # Select split
            df_split = df[df['split'] == split]
            # Loop over metrics
            for metric_name in metrics:
                # Filter metrics
                df_metric = df_split[df_split['metric'] == metric_name]
                # Consider last
                if split == 'test':
                    df_metric = df_metric.iloc[-2, :]
                else:
                    df_metric = df_metric.iloc[-1, :]
                row.append(df_metric['value'].item())
                cols_names.append(f"{metric_name}_{split}")
        vals.append(row)
            
#                 # Create row name
#                 row_name = f"{exp}_{dset_name}_{metric_name}"
#                 # Insert in df with new index
#                 vs.append(df_split['value'].set_axis(row_name, axis=0).rename(metric_name))
#                 values.insert(0, col_name, df_split['value'].set_axis(df_split['epoch'])) # works also
# Merge into a single df
# values = pd.concat(vs)#, axis=1)
# values
df_all = pd.DataFrame(vals, index=exps_names, columns=cols_names)
df_all

,loss_train,accuracy_train,precision_train,recall_train,f1score_train,specificity_train,iou_train,dice_train,hd_train,loss_validation,...,hd_validation,loss_test,accuracy_test,precision_test,recall_test,f1score_test,specificity_test,iou_test,dice_test,hd_test
20230507_bbbbd3_load_100k_7,0.813041,0.999620,0.995165,0.994817,0.994991,0.999809,0.990032,0.994991,8.731899,0.679647,...,79.771176,0.680478,0.986793,0.962245,0.867342,0.912332,0.997072,0.838796,0.912332,42.398873
20230512_load_bts4_4,0.822491,0.999603,0.994719,0.994821,0.994770,0.999792,0.989594,0.994770,14.950125,0.688977,...,128.934480,0.689550,0.983456,0.959103,0.826437,0.887842,0.996968,0.798305,0.887842,75.482203
20230512_load_bts5_5,0.819336,0.999540,0.994434,0.993437,0.993936,0.999781,0.987944,0.993936,5.930319,0.724605,...,102.321166,0.682159,0.980678,0.962486,0.786801,0.865822,0.997361,0.763391,0.865822,59.454097
20230514_load_bts_7,0.822726,0.999542,0.995910,0.991998,0.993950,0.999839,0.987973,0.993950,22.768195,0.686007,...,84.892443,0.691334,0.981765,0.964567,0.799218,0.874142,0.997474,0.776423,0.874142,42.867345
20230516_141_load_bts2_2,0.820035,0.999332,0.991246,0.991148,0.991197,0.999655,0.982548,0.991197,73.571939,0.690651,...,140.086507,0.703537,0.980203,0.963452,0.779722,0.861904,0.997455,0.757321,0.861904,95.566325


In [14]:
fmt = '{:10} {:<20} {:<10} {:<20}'
for col_name, values in df_all.items():
    mn = round(values.mean(), 4)
    std = round(values.std(), 4)
    print('{:>22}'.format(col_name), '{:>12}'.format(mn), '{:>2}'.format("+-"), '{:2}'.format(std))
#     print(fmt.format(col_name, mn, "+-", std))

            loss_train       0.8195 +- 0.0039
        accuracy_train       0.9995 +- 0.0001
       precision_train       0.9943 +- 0.0018
          recall_train       0.9932 +- 0.0017
         f1score_train       0.9938 +- 0.0015
     specificity_train       0.9998 +- 0.0001
             iou_train       0.9876 +- 0.003
            dice_train       0.9938 +- 0.0015
              hd_train      25.1905 +- 27.8089
       loss_validation        0.694 +- 0.0176
   accuracy_validation       0.9633 +- 0.0036
  precision_validation       0.9192 +- 0.0104
     recall_validation       0.6056 +- 0.0421
    f1score_validation       0.7296 +- 0.0318
specificity_validation       0.9953 +- 0.0005
        iou_validation       0.5751 +- 0.0406
       dice_validation       0.7296 +- 0.0318
         hd_validation     107.2012 +- 26.588
             loss_test       0.6894 +- 0.0092
         accuracy_test       0.9826 +- 0.0027
        precision_test       0.9624 +- 0.002
           recall_test       0.8119

## 